In [359]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [350]:
train_val = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [351]:
train_val.head(2)

,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
0,0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lyme_disease
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tungiasis


In [352]:
test.head(2)

,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,lymph_swells,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash
0,707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,708,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [353]:
X_train_val = train_val.drop(["id", "prognosis"], axis=1).to_numpy()
X_test = test.drop(["id"], axis=1).to_numpy()

In [354]:
cv = CountVectorizer()
y_train_val = cv.fit_transform(train_val.prognosis.to_numpy()).toarray()

In [355]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

In [356]:
# X_train = torch.tensor(X_train).float()
# X_val = torch.tensor(X_val).float()
# y_train = torch.tensor(y_train).float()
# y_val = torch.tensor(y_val).float()

# X_test = torch.tensor(X_test).float()

In [357]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape)

(636, 64) (636, 11)
(71, 64) (71, 11)
(303, 64)


In [403]:
# model = XGBClassifier()
model = LogisticRegression(max_iter=500)

In [404]:
model.fit(X_train, np.argmax(y_train, 1))

LogisticRegression(max_iter=500)

In [405]:
pred_test = model.predict_proba(X_test)

In [339]:
# in_features = X_train.shape[1]
# hidden_features = 10
# out_features = y_train.shape[1]

In [340]:
# device = "cpu"

In [326]:
# class NeuralNetwork(nn.Module):
#     def __init__(self, in_features, hidden_features, out_features):
#         super().__init__()
#         self.layer = nn.Sequential(
#             nn.Linear(in_features, hidden_features),
#             nn.ReLU(),
#             nn.Linear(hidden_features, hidden_features),
#             nn.ReLU(),
#             nn.Linear(hidden_features, out_features),
#             nn.Tanh()
#         )

#     def forward(self, x):
#         x = self.layer(x)
#         return x

# model = NeuralNetwork(in_features, hidden_features, out_features).to(device)
# print(model)

In [327]:
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [328]:
# epochs = 10
# for i in range(epochs):
#     pred = model(X_train)
#     loss = loss_fn(pred, y_train)

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     print(f"epoch: {i}, loss: {loss.item():.4f}")

In [329]:
# pred_val = torch.argmax(model(X_val), 1)

# accuracy_score(torch.argmax(y_val, 1), pred_val)

In [330]:
# pred_test = model(X_test)

In [406]:
pred_test.shape

(303, 11)

In [407]:
pred_ans = torch.topk(torch.tensor(pred_test), 3)[1].numpy()

In [408]:
# pred_ans

In [409]:
ans = [_.tolist() for _ in cv.inverse_transform(pred_ans)]

In [410]:
for idk in ans:
    for i in range(len(idk)):
        idk[i] = idk[i].capitalize()

In [411]:
# ans

In [412]:
final = [" ".join(_) for _ in ans]

In [413]:
len(final)

303

In [414]:
ans_df = pd.DataFrame({"id": test.id, "prognosis": final})

In [415]:
ans_df.head()

,id,prognosis
0,707,Chikungunya Dengue Japanese_encephalitis
1,708,Chikungunya Japanese_encephalitis
2,709,Chikungunya Dengue Japanese_encephalitis
3,710,Chikungunya Dengue Japanese_encephalitis
4,711,Chikungunya Dengue Japanese_encephalitis


In [416]:
ans_df.to_csv("outputs/ans3.csv", index=False)